In [ ]:
import numpy as np
import pandas as pd 
import os
print(os.listdir("../input"))

In [ ]:
breeds = pd.read_csv('../input/breed_labels.csv')
colors = pd.read_csv('../input/color_labels.csv')
states = pd.read_csv('../input/state_labels.csv')
train = pd.read_csv('../input/train/train.csv')
test = pd.read_csv('../input/test/test.csv')

train.head()

1. колонка 'name' - вероятно, что если в этом признаке написано "no name yet" значит у животного еще не было хозяина, если есть имя, то очевидно у животного был хозяин
1. 'Age' - возраст (порядковая фича)
1. колонка 'breed1' - вероятно, это код породы животного (категориальная)
1. колонка 'breed2' уточнение породы (если порода смешеная)
1. колонка "пол" - определяет пол животного
1. Колонка цвет1, цвет2, цвет3
1. Колонка MaturitySize - размер животного во взрослом возрасте
1. колонка FurLength - длинна шерсти (1 короткая,2 средняя, 3 длинная)
1. колонка Vaccinated - есть ли прививки 1 да, 2 нет, 3 не уверены
1. колонка Dewormed - наличие паразитов 1 да, 2 нет, 3 не уверены
1. колонка Sterilized - стирилизация животного 1 да, 2 нет, 3 не уверены
1. колонка Health - состояние здоровья (1 хорошее, 2 не очень, 3 плохое, 0 не определено)
1. колонка Quantity - количество животных
1. колонка fee - плата за усыновление
1. колонка State - штат в малазии
1. колонка RescuerID - айди животного
1. колонка VideoAmt видео животного
1. колонка PhotoAmt фото животного
1. колонка Description описание животного

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from PIL import Image
import gc

In [ ]:
print("train.csv shape is {}".format(train.shape))
print("test.csv shape is {}".format(test.shape))

In [ ]:
fig, ax = plt.subplots(figsize=(6, 5))
ax.set_title("AdoptionSpeed count in train")
sns.countplot(x="AdoptionSpeed", data=train, ax=ax)

Смотрим, как распределены порядковые данные

In [ ]:
sns.distplot(train['Age'], bins = 100, kde=False)

In [ ]:
train.Age.unique()

In [ ]:
#Удаляю средний возраст больше 20 и заменяю средним значением
train['Age'] = train['Age'].fillna( method='pad')
train.loc[train['Age'] > 20, 'Age'] = train['Age'].mean()

In [ ]:
sns.distplot(train['Age'], bins = 20, kde=False)

In [ ]:
breed_dict = dict(zip(breeds['BreedID'], breeds['BreedName']))

In [ ]:
animal_dict = {1: 'Dog', 2: 'Cat'}

In [ ]:
train.head()

In [ ]:
import warnings
warnings.filterwarnings("ignore", 'This pattern has match groups')

def make_features(train):
    train['Named'] = (~train['Name'].isnull() | train['Name'].str.contains('(No | Puppies | Kitty | Puppy | Kitten)')).astype(int)
    train['PureBreed'] = (train.Breed2==0 | ~train.Breed1.isin(breeds[breeds.BreedName.str.contains('Domestic' or 'Mixed')].BreedID.values)).astype(int)
    train = train.drop('Breed2', axis=1)

    train = train.select_dtypes(exclude=[object])
    return train

dealed_train = make_features(train)

In [ ]:
dealed_train['Type'] = dealed_train['Type'].map(animal_dict)
dealed_train['Breed1_new'] = dealed_train['Breed1'].map(breed_dict)

In [ ]:
#Почему-то после применения функции make_features у меня пропадает PetID
#Поэтому я пойду вот на таких костылях
train_2 =  dealed_train.combine_first(train)
train = train_2.drop('Breed1', axis=1)
train = train.drop('Breed2', axis=1)
train = train.drop('Named', axis=1)

In [ ]:
#Делаю аналогичные преображения с тестовой выборкой
dealed_test = make_features(test)
dealed_test['Type'] = dealed_test['Type'].map(animal_dict)
dealed_test['Breed1_new'] = dealed_test['Breed1'].map(breed_dict)
test_2 =  dealed_test.combine_first(test)
test = test_2.drop('Breed1', axis=1)
test = test.drop('Breed2', axis=1)
test = test.drop('Named', axis=1)

In [ ]:
train_dogs = train[dealed_train['Type'] == 'Dog']
train_cats = train[dealed_train['Type'] == 'Cat']

In [ ]:
plt.figure(figsize=(28, 12));
sns.barplot(x='PhotoAmt', y='AdoptionSpeed', data=train)
plt.title('What about photos?')

In [ ]:
#Создаем для кошек датафреймы для описания распределений скорости adoption в разрезах различных категориальных фич
ms_train_cats = pd.DataFrame(train_cats.groupby([ 'AdoptionSpeed', 'MaturitySize'])['PetID'].count()).reset_index()
fr_train_cats = pd.DataFrame(train_cats.groupby(['AdoptionSpeed', 'FurLength'])['PetID'].count()).reset_index()
vc_train_cats = pd.DataFrame(train_cats.groupby(['AdoptionSpeed', 'Vaccinated'])['PetID'].count()).reset_index()
dw_train_cats = pd.DataFrame(train_cats.groupby(['AdoptionSpeed', 'Dewormed'])['PetID'].count()).reset_index()
st_train_cats = pd.DataFrame(train_cats.groupby(['AdoptionSpeed', 'Sterilized'])['PetID'].count()).reset_index()
h_train_cats = pd.DataFrame(train_cats.groupby(['AdoptionSpeed', 'Health'])['PetID'].count()).reset_index()
age_train_cats = pd.DataFrame(train_cats.groupby(['AdoptionSpeed', 'Age'])['PetID'].count()).reset_index()

In [ ]:
#Добавляем колонку в датафреймы с разрезами с долей питомцев от общего числа для каждого значения категориальной фичи
ms_train_cats['PetID_Share'] = ms_train_cats['PetID']/14993
fr_train_cats['PetID_Share'] = ms_train_cats['PetID']/14993
vc_train_cats['PetID_Share'] = ms_train_cats['PetID']/14993
dw_train_cats['PetID_Share'] = ms_train_cats['PetID']/14993
st_train_cats['PetID_Share'] = ms_train_cats['PetID']/14993
h_train_cats['PetID_Share'] = ms_train_cats['PetID']/14993
age_train_cats['PetID_Share'] = ms_train_cats['PetID']/14993
#Если честно, не понимаю, для чего были совершены эти действия и предлагаю это потом удалить ))

In [ ]:
#распределение размеров по количеству питомцев
train_cats.groupby(['MaturitySize'])['PetID'].count()

In [ ]:
sns.distplot(train_cats['MaturitySize'])

In [ ]:
#распределение размеров питомцев в разрезе adoption скорости
train_cats.groupby(['MaturitySize', 'AdoptionSpeed'])['PetID'].count()

In [ ]:
# Хитмап по размеру питомца
pv_ms_train_cats = ms_train_cats.pivot_table(values='PetID_Share', index='MaturitySize',   columns='AdoptionSpeed')
sns.heatmap(pv_ms_train_cats, cmap='inferno_r')

Encode some features

In [ ]:
# Утащила у каких-то студентиков https://www.kaggle.com/tgibbons/student-project-looking-for-feedback
encodedColor1 = pd.get_dummies( train['Color1'], prefix="color" )

# Add the new dummy variables to the pet_train data frame
train = pd.concat([train, encodedColor1], axis='columns')
# Do the same thing to the submission data
encodedColor2 = pd.get_dummies( test['Color1'], prefix="color" )
test = pd.concat([test, encodedColor2], axis='columns')

# print out the current data
print ("Size of pet_train = ", train.shape)
print ("Size of pet_submit = ", test.shape)
train.head(5)

In [ ]:
cat_columns = ['Breed1_new','FurLength','Dewormed']

# Create the dummy variables for the columns listed above
dfTemp = pd.get_dummies( train[cat_columns], columns=cat_columns )
train = pd.concat([train, dfTemp], axis='columns')

# Do the same to the submission data
dfSummit = pd.get_dummies( test[cat_columns], columns=cat_columns )
test = pd.concat([test, dfSummit], axis='columns')
# Get missing columns in the submission data
missing_cols = set( train.columns ) - set( test.columns )
# Add a missing column to the submission set with default value equal to 0
for c in missing_cols:
    test[c] = 0
# Ensure the order of column in the test set is in the same order than in train set
test = test[train.columns]




# print out the current data
print ("Size of pet_train = ", train.shape)
print ("Size of pet_submit = ", test.shape)
train.head(5)

In [ ]:
import cv2
import pandas as pd
import numpy as np
import os
from tqdm import tqdm, tqdm_notebook
from keras.applications.densenet import preprocess_input, DenseNet121

img_size = 256
batch_size = 16

In [ ]:
pet_ids = train['PetID'].values
n_batches = len(pet_ids) // batch_size + 1

In [ ]:
# The following functions and the rest solution have been taken from Valentina's kernel
# Thank you <3
def confusion_matrix(rater_a, rater_b, min_rating=None, max_rating=None):
    """
    Returns the confusion matrix between rater's ratings
    """
    assert(len(rater_a) == len(rater_b))
    if min_rating is None:
        min_rating = min(rater_a + rater_b)
    if max_rating is None:
        max_rating = max(rater_a + rater_b)
    num_ratings = int(max_rating - min_rating + 1)
    conf_mat = [[0 for i in range(num_ratings)]
                for j in range(num_ratings)]
    for a, b in zip(rater_a, rater_b):
        conf_mat[a - min_rating][b - min_rating] += 1
    return conf_mat


def histogram(ratings, min_rating=None, max_rating=None):
    """
    Returns the counts of each type of rating that a rater made
    """
    if min_rating is None:
        min_rating = min(ratings)
    if max_rating is None:
        max_rating = max(ratings)
    num_ratings = int(max_rating - min_rating + 1)
    hist_ratings = [0 for x in range(num_ratings)]
    for r in ratings:
        hist_ratings[r - min_rating] += 1
    return hist_ratings


def quadratic_weighted_kappa(y, y_pred):
    """
    Calculates the quadratic weighted kappa
    axquadratic_weighted_kappa calculates the quadratic weighted kappa
    value, which is a measure of inter-rater agreement between two raters
    that provide discrete numeric ratings.  Potential values range from -1
    (representing complete disagreement) to 1 (representing complete
    agreement).  A kappa value of 0 is expected if all agreement is due to
    chance.
    quadratic_weighted_kappa(rater_a, rater_b), where rater_a and rater_b
    each correspond to a list of integer ratings.  These lists must have the
    same length.
    The ratings should be integers, and it is assumed that they contain
    the complete range of possible ratings.
    quadratic_weighted_kappa(X, min_rating, max_rating), where min_rating
    is the minimum possible rating, and max_rating is the maximum possible
    rating
    """
    rater_a = y
    rater_b = y_pred
    min_rating=None
    max_rating=None
    rater_a = np.array(rater_a, dtype=int)
    rater_b = np.array(rater_b, dtype=int)
    assert(len(rater_a) == len(rater_b))
    if min_rating is None:
        min_rating = min(min(rater_a), min(rater_b))
    if max_rating is None:
        max_rating = max(max(rater_a), max(rater_b))
    conf_mat = confusion_matrix(rater_a, rater_b,
                                min_rating, max_rating)
    num_ratings = len(conf_mat)
    num_scored_items = float(len(rater_a))

    hist_rater_a = histogram(rater_a, min_rating, max_rating)
    hist_rater_b = histogram(rater_b, min_rating, max_rating)

    numerator = 0.0
    denominator = 0.0

    for i in range(num_ratings):
        for j in range(num_ratings):
            expected_count = (hist_rater_a[i] * hist_rater_b[j]
                              / num_scored_items)
            d = pow(i - j, 2.0) / pow(num_ratings - 1, 2.0)
            numerator += d * conf_mat[i][j] / num_scored_items
            denominator += d * expected_count / num_scored_items

    return (1.0 - numerator / denominator)

In [ ]:
# Кажется, это адский алгоритм для вычисления этой самой каппы
# Но я ни в чем не уверена
class OptimizedRounder(object):
    def __init__(self):
        self.coef_ = 0

    def _kappa_loss(self, coef, X, y):
        X_p = np.copy(X)
        for i, pred in enumerate(X_p):
            if pred < coef[0]:
                X_p[i] = 0
            elif pred >= coef[0] and pred < coef[1]:
                X_p[i] = 1
            elif pred >= coef[1] and pred < coef[2]:
                X_p[i] = 2
            elif pred >= coef[2] and pred < coef[3]:
                X_p[i] = 3
            else:
                X_p[i] = 4

        ll = quadratic_weighted_kappa(y, X_p)
        return -ll

    def fit(self, X, y):
        loss_partial = partial(self._kappa_loss, X=X, y=y)
        initial_coef = [0.5, 1.5, 2.5, 3.5]
        self.coef_ = sp.optimize.minimize(loss_partial, initial_coef, method='nelder-mead')

    def predict(self, X, coef):
        X_p = np.copy(X)
        for i, pred in enumerate(X_p):
            if pred < coef[0]:
                X_p[i] = 0
            elif pred >= coef[0] and pred < coef[1]:
                X_p[i] = 1
            elif pred >= coef[1] and pred < coef[2]:
                X_p[i] = 2
            elif pred >= coef[2] and pred < coef[3]:
                X_p[i] = 3
            else:
                X_p[i] = 4
        return X_p

    def coefficients(self):
        return self.coef_['x']
    
def rmse(actual, predicted):
    return sqrt(mean_squared_error(actual, predicted))

In [ ]:
#Что здесь происходит?
#Мне страшно
from keras.models import Model
from keras.layers import GlobalAveragePooling2D, Input, Lambda, AveragePooling1D
import keras.backend as K
from keras.applications.densenet import DenseNet121
inp = Input((256,256,3))
backbone = DenseNet121(input_tensor = inp, 
                       weights=None, #Мне не удалось загрузить весы, которые были у Валентины в отдельном файле, поэтому пускай алгоритм сам учится
                       include_top = False)
x = backbone.output
x = GlobalAveragePooling2D()(x)
x = Lambda(lambda x: K.expand_dims(x,axis = -1))(x)
x = AveragePooling1D(4)(x)
out = Lambda(lambda x: x[:,:,0])(x)

m = Model(inp,out)


In [ ]:
#Это функция, которая превращает изображения в квадратики?? Серьезно?
#Но зачем?
def resize_to_square(im):
    old_size = im.shape[:2] # old_size is in (height, width) format
    ratio = float(img_size)/max(old_size)
    new_size = tuple([int(x*ratio) for x in old_size])
    # new_size should be in (width, height) format
    im = cv2.resize(im, (new_size[1], new_size[0]))
    delta_w = img_size - new_size[1]
    delta_h = img_size - new_size[0]
    top, bottom = delta_h//2, delta_h-(delta_h//2)
    left, right = delta_w//2, delta_w-(delta_w//2)
    color = [0, 0, 0]
    new_im = cv2.copyMakeBorder(im, top, bottom, left, right, cv2.BORDER_CONSTANT,value=color)
    return new_im

def load_image(path, pet_id):
    image = cv2.imread(f'{path}{pet_id}-1.jpg')
    new_image = resize_to_square(image)
    new_image = preprocess_input(new_image)
    return new_image

In [ ]:
features = {}
for b in tqdm_notebook(range(n_batches)):
    start = b*batch_size
    end = (b+1)*batch_size
    batch_pets = pet_ids[start:end]
    batch_images = np.zeros((len(batch_pets),img_size,img_size,3))
    for i,pet_id in enumerate(batch_pets):
        try:
            batch_images[i] = load_image("../input/train_images/", pet_id)
        except:
            pass
    batch_preds = m.predict(batch_images)
    for i,pet_id in enumerate(batch_pets):
        features[pet_id] = batch_preds[i]
#К сожалению, мне не удалось удалить дубли изображений, тк у макбуков какая-то проблема с CUDA
#Или у меня какая-то проблема с CUDA
#Поэтому все будет долго и мучительно

In [ ]:
train_feats = pd.DataFrame.from_dict(features, orient='index')
train_feats.columns = ['pic_'+str(i) for i in range(train_feats.shape[1])]

In [ ]:
pet_ids = test['PetID'].values
n_batches = len(pet_ids) // batch_size + 1

features = {}
for b in tqdm_notebook(range(n_batches)):
    start = b*batch_size
    end = (b+1)*batch_size
    batch_pets = pet_ids[start:end]
    batch_images = np.zeros((len(batch_pets),img_size,img_size,3))
    for i,pet_id in enumerate(batch_pets):
        try:
            batch_images[i] = load_image("../input/petfinder-adoption-prediction/test_images/", pet_id)
        except:
            pass
    batch_preds = m.predict(batch_images)
    for i,pet_id in enumerate(batch_pets):
        features[pet_id] = batch_preds[i]

In [ ]:
test_feats = pd.DataFrame.from_dict(features, orient='index')
test_feats.columns = ['pic_'+str(i) for i in range(test_feats.shape[1])]

In [ ]:
test_feats = test_feats.reset_index()
test_feats.rename({'index': 'PetID'}, axis='columns', inplace=True)

train_feats = train_feats.reset_index()
train_feats.rename({'index': 'PetID'}, axis='columns', inplace=True)

test_feats.head()

In [ ]:
target = train['AdoptionSpeed']
train_id = train['PetID']
test_id = test['PetID']


train = pd.merge(train, train_feats, how='left', on='PetID')
test = pd.merge(test, test_feats, how='left', on='PetID')

train.drop(['AdoptionSpeed', 'PetID'], axis=1, inplace=True)
test.drop(['PetID'], axis=1, inplace=True)

In [ ]:
numeric_cols = ['Age', 'Quantity', 'Fee', 'VideoAmt', 'PhotoAmt', 'AdoptionSpeed', 
                'doc_sent_mag', 'doc_sent_score', 'dominant_score', 'dominant_pixel_frac', 
                'dominant_red', 'dominant_green', 'dominant_blue', 'bounding_importance', 
                'bounding_confidence', 'vertex_x', 'vertex_y', 'label_score'] +\
               [col for col in train.columns if col.startswith('pic') or col.startswith('svd')]
cat_cols = list(set(train.columns) - set(numeric_cols))
train.loc[:, cat_cols] = train[cat_cols].astype('category')
test.loc[:, cat_cols] = test[cat_cols].astype('category')
print(train.shape)
print(test.shape)

# get the categorical features
foo = train.dtypes
cat_feature_names = foo[foo == "category"]
cat_features = [train.columns.get_loc(c) for c in train.columns if c in cat_feature_names]

In [ ]:
from sklearn.model_selection import StratifiedKFold
import lightgbm as lgb
from functools import partial
import scipy as sp
from collections import Counter
from math import sqrt
from sklearn.metrics import cohen_kappa_score, mean_squared_error

In [ ]:
N_SPLITS = 5
def run_cv_model(train, test, target, model_fn, params={}, eval_fn=None, label='model'):
    kf = StratifiedKFold(n_splits=N_SPLITS, random_state=42, shuffle=True)
    fold_splits = kf.split(train, target)
    cv_scores = []
    qwk_scores = []
    pred_full_test = 0
    pred_train = np.zeros((train.shape[0], N_SPLITS))
    all_coefficients = np.zeros((N_SPLITS, 4))
    feature_importance_df = pd.DataFrame()
    i = 1
    for dev_index, val_index in fold_splits:
        print('Started ' + label + ' fold ' + str(i) + '/' + str(N_SPLITS))
        if isinstance(train, pd.DataFrame):
            dev_X, val_X = train.iloc[dev_index], train.iloc[val_index]
            dev_y, val_y = target[dev_index], target[val_index]
        else:
            dev_X, val_X = train[dev_index], train[val_index]
            dev_y, val_y = target[dev_index], target[val_index]
        params2 = params.copy()
        pred_val_y, pred_test_y, importances, coefficients, qwk = model_fn(dev_X, dev_y, val_X, val_y, test, params2)
        pred_full_test = pred_full_test + pred_test_y
        pred_train[val_index] = pred_val_y
        all_coefficients[i-1, :] = coefficients
        if eval_fn is not None:
            cv_score = eval_fn(val_y, pred_val_y)
            cv_scores.append(cv_score)
            qwk_scores.append(qwk)
            print(label + ' cv score {}: RMSE {} QWK {}'.format(i, cv_score, qwk))
        fold_importance_df = pd.DataFrame()
        fold_importance_df['feature'] = train.columns.values
        fold_importance_df['importance'] = importances
        fold_importance_df['fold'] = i
        feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)        
        i += 1
    print('{} cv RMSE scores : {}'.format(label, cv_scores))
    print('{} cv mean RMSE score : {}'.format(label, np.mean(cv_scores)))
    print('{} cv std RMSE score : {}'.format(label, np.mean(cv_scores)))
    print('{} cv QWK scores : {}'.format(label, qwk_scores))
    print('{} cv mean QWK score : {}'.format(label, np.mean(qwk_scores)))
    print('{} cv std QWK score : {}'.format(label, np.std(qwk_scores)))
    pred_full_test = pred_full_test / float(N_SPLITS)
    results = {'label': label,
               'train': pred_train, 'test': pred_full_test,
                'cv': cv_scores, 'qwk': qwk_scores,
               'importance': feature_importance_df,
               'coefficients': all_coefficients}
    return results

params = {'application': 'regression',
          'boosting': 'gbdt',
          'metric': 'rmse',
          'num_leaves': 70,
          'max_depth': 9,
          'learning_rate': 0.01,
          'bagging_fraction': 0.85,
          'feature_fraction': 0.8,
          'min_split_gain': 0.02,
          'min_child_samples': 150,
          'min_child_weight': 0.02,
          'lambda_l2': 0.0475,
          'verbosity': -1,
          'data_random_seed': 17,
          'early_stop': 600,
          'verbose_eval': 100,
          'num_rounds': 10000}

def runLGB(train_X, train_y, test_X, test_y, test_X2, params):
    print('Prep LGB')
    d_train = lgb.Dataset(train_X, label=train_y)
    d_valid = lgb.Dataset(test_X, label=test_y)
    watchlist = [d_train, d_valid]
    print('Train LGB')
    num_rounds = params.pop('num_rounds')
    verbose_eval = params.pop('verbose_eval')
    early_stop = None
    if params.get('early_stop'):
        early_stop = params.pop('early_stop')
    model = lgb.train(params,
                      train_set=d_train,
                      num_boost_round=num_rounds,
                      valid_sets=watchlist,
                      verbose_eval=verbose_eval,
                      categorical_feature=list(cat_features),
                      early_stopping_rounds=early_stop)
    
    print('Predict 1/2')
    pred_test_y = model.predict(test_X, num_iteration=model.best_iteration)
    optR = OptimizedRounder()
    optR.fit(pred_test_y, test_y)
    coefficients = optR.coefficients()
    pred_test_y_k = optR.predict(pred_test_y, coefficients)
    print("Valid Counts = ", Counter(test_y))
    print("Predicted Counts = ", Counter(pred_test_y_k))
    print("Coefficients = ", coefficients)
    qwk = quadratic_weighted_kappa(test_y, pred_test_y_k)
    print("QWK = ", qwk)
    print('Predict 2/2')
    pred_test_y2 = model.predict(test_X2, num_iteration=model.best_iteration)
    return pred_test_y.reshape(-1, 1), pred_test_y2.reshape(-1, 1), model.feature_importance(), coefficients, qwk

results = run_cv_model(train, test, target, runLGB, params, rmse, 'lgb')

In [ ]:
imports = results['importance'].groupby('feature')['feature', 'importance'].mean().reset_index()
imports.sort_values('importance', ascending=False)
#Видела, что Валентина вырезала идентификатор человека, у которого находится животное. Не знаю, почему. Я пока оставлю

In [ ]:
optR = OptimizedRounder()
coefficients_ = np.mean(results['coefficients'], axis=0)
print(coefficients_)
# manually adjust coefs
coefficients_[0] = 1.645
coefficients_[1] = 2.115
coefficients_[3] = 2.84
train_predictions = [r[0] for r in results['train']]
train_predictions = optR.predict(train_predictions, coefficients_).astype(int)
Counter(train_predictions)

In [ ]:
optR = OptimizedRounder()
coefficients_ = np.mean(results['coefficients'], axis=0)
print(coefficients_)
# manually adjust coefs
coefficients_[0] = 1.645
coefficients_[1] = 2.115
coefficients_[3] = 2.84
test_predictions = [r[0] for r in results['test']]
test_predictions = optR.predict(test_predictions, coefficients_).astype(int)
Counter(test_predictions)

In [ ]:
print("True Distribution:")
print(pd.value_counts(target, normalize=True).sort_index())
print("Test Predicted Distribution:")
print(pd.value_counts(test_predictions, normalize=True).sort_index())
print("Train Predicted Distribution:")
print(pd.value_counts(train_predictions, normalize=True).sort_index())

In [ ]:
quadratic_weighted_kappa(target, train_predictions)
rmse(target, [r[0] for r in results['train']])
submission = pd.DataFrame({'PetID': test_id, 'AdoptionSpeed': test_predictions})
submission.head()

In [ ]:
submission.to_csv('submission.csv', index=False)

Итак, я добавила некоторые фичи, затем сделала предсказание по алгоритму Валентины и сохранила результат
Как вариант, мне кажется, ты можешь попробовать сделать предсказание нейросетью, записать ее результат в какой-нибудь сабмишн1, а потом открыть оба файла и вывести для каждого петАйди среднее значение.
Кажется, я видела методы, которые могут это, в этой презентации: https://alexanderdyakonov.files.wordpress.com/2015/04/ama2015_pandas.pdf
Ну или сделать что-нибудь другое )))